In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


URL = 'https://finviz.com/quote.ashx?t='
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}


def get_html(url, params=None):
  return requests.get(url, headers=HEADERS, params=params)


def manfct(el):
  if el == '-':
    return 'NaN'
  elif el[-1] == '%':
    return float(el[:-1])  
  else:
    return float(el)


def get_industry(html):
  soup = BeautifulSoup(html, 'html.parser')
  data = soup.find_all('table', class_='snapshot-table2')

  df = pd.read_html(str(data[0]))[0]

  recom = manfct(df.iloc[11, 1])

  pe = manfct(df.iloc[0, 3])
  fpe = manfct(df.iloc[1, 3])
  peg = manfct(df.iloc[2, 3])
  debt = manfct(df.iloc[9, 3])
  sma50 = manfct(df.iloc[11, 5])
  roe = manfct(df.iloc[5, 7])
  sma200 = manfct(df.iloc[11, 7])
  target = manfct(df.iloc[4, 9])
  shortFt = manfct(df.iloc[2, 9])
  rsi = manfct(df.iloc[8, 9])
  current = manfct(df.iloc[10, 11])
  if sma50 != 'NaN' and sma200 != 'NaN':
    sma = sma50 - sma200
  else:
    sma = 'NaN'
  
  if target != 'NaN' and current != 'NaN':
    fct = round((target / current - 1) * 100, 2)
  else:
    fct = 'NaN'

  return recom, pe, fpe, peg, debt, roe, shortFt, rsi, sma, fct


def parse(tic):
  html = get_html(URL + tic + '&ty=c&ta=1&p=d')
  if html.status_code == 200:
    print(tic)
    recom, pe, fpe, peg, debt, roe, shortFt, rsi, sma, fct = get_industry(html.text)
    return tic, recom, pe, fpe, peg, debt, roe, shortFt, rsi, sma, fct
  else:
    print(tic)
    print('Error')
    return tic, 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN' ,'NaN' ,'NaN'


df = pd.read_csv('/spb_stocks.csv')
ticker = list(df['ticker'])

data = []
for tic in ticker:
  data.append(parse(tic))

print(data)

A
AA
AAL
AAN
AAP
AAPL
AAXN
ABBV
ABC
ABG
ABMD
ABT
ACAD
ACIA
ACM
ACN
ADBE
ADI
ADM
ADP
ADS
ADSK
ADUS
AEE
AEIS
AERI
AES
AFG
AFL
AGCO
AGIO
AIG
AIMT
AIR
AIV
AIZ
AJG
AJRD
AKAM
AKZM
Error
ALB
ALGN
ALGT
ALK
ALL
ALLE
ALLK
ALLO
ALNY
ALRM
ALSN
ALTR
ALV
ALXN
AMAT
AMCX
AMD
AME
AMED
AMG
AMGN
AMN
AMP
AMT
AMWD
AMZN
AN
ANAB
ANDE
ANET
ANGI
ANIK
ANIP
ANSS
ANTM
AOBC
AON
AOS
APA
APD
APEI
APH
APLE
APPF
APPN
APTV
APY
ARE
ARMK
ARNA
ARNC
ARW
ARWR
ASGN
ASH
ASIX
ATGE
ATKR
ATR
ATRA
ATRI
ATRO
ATUS
ATVI
AVAV
AVB
AVGO
AVLR
AVNS
AVY
AWI
AWK
AWR
AX
AXE
AXGN
AXP
AXSM
AYI
AYX
AZO
AZPN
BA
BAC
BAH
BAND
BAST
Error
BAX
BBBY
BBSI
BBY
BC
BCO
BCPC
BDC
BDX
BEAT
BECN
BEN
BERY
BF.B
Error
BFAM
BFYT
BH
BHF
BIG
BIIB
BIO
BJRI
BK
BKI
BKNG
BKR
BL
BLD
BLDR
BLK
BLKB
BLL
BLUE
BMCH
BMI
BMRN
BMW
Error
BMY
BOH
BOKF
BOOT
BPMC
BR
BRC
BRK.B
Error
BRKR
BRO
BSX
BURL
BWA
BWXT
BXP
BYND
C
CABO
CACC
CAG
CAH
CALM
CARA
CARG
CARR
CARS
CASY
CAT
CB
CBPO
CBRE
CBRL
CBSH
CBU
CCI
CCK
CCL
CCMP
CDK
CDNA
CDNS
CDW
CE
CENT
CENTA
CERN
CF
CFG
CFR
CFX
CGNX
CHD
CHDN
CH

In [0]:
df = pd.DataFrame(data, columns=['ticker', 'recomendation', 'P/E', 'future P/E', 'PEG', 'Debt/Equity', 'ROE', 'shorts, %', 'RSI', 'SMA50 - SMA200, %', 'forecast FINVIZ, %'])
df.to_csv('/indicators.csv', index=False)